In [ ]:
%%capture
!pip install eccodes
!pip install ecmwflibs
!pip install cfgrib
!python -m cfgrib selfcheck
!pip install xarray
import cfgrib, xarray as xr, numpy as np, pandas as pd
import pyarrow.feather as feather

!pip install datashader
import datashader as das

!pip install opencv-python s3fs netCDF4
import netCDF4, requests, s3fs, fnmatch, cv2

# toa radiation

In [ ]:
path = 'gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/sfc-dgg/tsr/tsr_178_h3f7-ndjf_000900.sfc-dgg.O8000.global.grib'
ds = xr.open_dataset(path, engine='cfgrib', indexpath='')
df = pd.DataFrame({'lat':ds.latitude.values, 'lon':ds.longitude.values, 'tsr':ds.tsr.values})
df = df[df.lon.between(360-123,360-68)]
df = df[df.lat.between(-50,5)]
df.reset_index().to_feather(f'test_tsr_2018-11-01T15.arrow')

path = 'gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/sfc-dgg/tisr/tisr_212_h3f7-ndjf_000900.sfc-dgg.O8000.global.grib'
ds = xr.open_dataset(path, engine='cfgrib', indexpath='')
df = pd.DataFrame({'lat':ds.latitude.values, 'lon':ds.longitude.values, 'tisr':ds.tisr.values})
df = df[df.lon.between(360-123,360-68)]
df = df[df.lat.between(-50,5)]
df.reset_index().to_feather(f'test_tisr_2018-11-01T15.arrow')

# GOES BAND AGGREGATIONS

In [ ]:
### goes funtions for lat lon and xy slicing ###
def get_lat_lon(ds):
    x,y = np.meshgrid(ds.x,ds.y)
    goes_imager_projection = ds.goes_imager_projection
    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat
    
    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2
    r_s = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)
    
    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)
    
    ds = ds.assign_coords({"lat":(["y","x"],lat),"lon":(["y","x"],lon)})
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds
    
dates = pd.date_range('2018-11-01','2018-11-01')
date = dates[0]
hour = 18
datetime = date + pd.Timedelta(hour, 'Hours')
fs = s3fs.S3FileSystem( anon = True )

files = np.array( fs.ls(f'noaa-goes16/ABI-L1b-RadF/{date.year}/{date.dayofyear:03.0f}/{hour:02.0f}/') ) # goes16 is from 2017-059, goes17 is 2018-240, goes18 is 2022-210 
out = np.zeros((10000,10000), dtype=np.float32)
for bi, band in enumerate(range(1,8)):
    bandfiles = fnmatch.filter(files, f'*C0{band}*')
    key = bandfiles[0][12:]
    resp = requests.get( f'https://noaa-goes16.s3.amazonaws.com/{key}' )
    file_name = key.split('/')[-1].split('.')[0]
    store = netCDF4.Dataset(file_name, memory = resp.content)
    ds = xr.open_dataset( xr.backends.NetCDF4DataStore(store) )
    if band == 2: ds = ds.coarsen(x=2,y=2, boundary='trim').mean()
    else: pass
    if band == 1: 
        ds = get_lat_lon(ds)
        lat = cv2.resize(ds.lat.values, (10000,10000))
        lon = cv2.resize(ds.lon.values, (10000,10000))
    else: pass
    out += cv2.resize(ds.Rad.values, (10000,10000))

df = pd.DataFrame({'lat':lat.ravel(), 'lon':lon.ravel(), 'rad':out.ravel()})
df = df[df.lon.between(-123,-68)]
df = df[df.lat.between(-50,5)]
df.to_parquet(f'GOES_{hour}.parquet', engine='auto', compression='snappy', index=None)

# to get cloud liquid water path 

In [ ]:
ts = '001980'
path = f'/gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/ml-dgg/clwc/clwc_246_h3f7-ndjf_{ts}.ml-dgg.O8000.global.grib'
ds1 = xr.open_dataset(path, engine='cfgrib', indexpath='')

levels = pd.read_csv('137_levels_table.csv')
levels.Density = pd.to_numeric(levels.Density, errors='coerce')
levels.Geometric_Altitude = pd.to_numeric(levels.Geometric_Altitude, errors='coerce')
height = np.pad(levels.Geometric_Altitude.values, (0, 1), 'constant')

out = np.zeros(256288000, dtype=np.float32)
for i in range(75,138):
    print(i)
    h1 = (height[i-1] + height[i]) / 2
    h2 = (height[i] + height[i+1]) / 2
    out += ds1.sel(hybrid=i, method='nearest').clwc.values * levels.Density[i] * (h1-h2)

df = pd.DataFrame({'lat':ds1.latitude.values, 'lon':ds1.longitude.values, 'clwp':out})
df = df[df.lon.between(360-123,360-68)]
df = df[df.lat.between(-50,5)]
df.reset_index().to_feather(f'all_clwc_{str(ds1.time.values)[:14]}.arrow')

In [ ]:
ts = '001980'
path1 = f'/gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/ml-dgg/clwc/clwc_246_h3f7-ndjf_{ts}.ml-dgg.O8000.global.grib'
path2 = f'/gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/ml-dgg/t/t_130_h3f7-ndjf_{ts}.ml-dgg.O8000.global.grib'
path3 = f'/gpfs/alpine/cli147/proj-shared/scratch/data/h3f7-ndjf/global/O8000/ml-dgg/pres/pres_54_h3f7-ndjf_{ts}.ml-dgg.O8000.global.grib'
ds1 = xr.open_dataset(path1, engine='cfgrib', indexpath='')
print('done')
ds2 = xr.open_dataset(path2, engine='cfgrib', indexpath='')
print('done')
ds3 = xr.open_dataset(path3, engine='cfgrib', indexpath='')
print('done')

levels = pd.read_csv('137_levels_table.csv')
levels.Density = pd.to_numeric(levels.Density, errors='coerce')
levels.Geometric_Altitude = pd.to_numeric(levels.Geometric_Altitude, errors='coerce')
height = levels.Geometric_Altitude.values

out = np.zeros(256288000, dtype=np.float32)
for i in range(75,138):
    print(i)
    h1 = (height[i-1] + height[i]) / 2
    h2 = (height[i] + height[i+1]) / 2
    out += ds1.sel(hybrid=i, method='nearest').clwc.values * levels.Density[i] * (h1-h2)

df = pd.DataFrame({'lat':ds1.latitude.values, 'lon':ds1.longitude.values, 'clwp':out})
df = df[df.lon.between(360-123,360-68)]
df = df[df.lat.between(-50,5)]
df.reset_index().to_feather(f'all_clwc_{str(ds1.time.values)[:14]}.arrow')

In [ ]:
import pyarrow as pa, pyarrow.parquet as pq
for i in range(75,138):
    print(i)
    temp = ds1.sel(hybrid=i, method='nearest').clwc.values * ds3.sel(hybrid=i, method='nearest').pres.values  / (287 * ds2.sel(hybrid=i, method='nearest').t.values)
    table = pa.Table.from_arrays([ temp ], names=['clwp'])
    pq.write_table(table, f'{i:03}_clwc.parquet')
    
out = np.zeros(256288000, dtype=np.float32)
for i in range(75,138):
    print(i)
    temp = pq.read_table(f'{i:03}_clwc.parquet')
    out += np.array(temp['clwp'])

df = pd.DataFrame({'lat':ds1.latitude.values, 'lon':ds1.longitude.values, 'clwp':out})
df = df[df.lon.between(360-123,360-68)]
df = df[df.lat.between(-50,5)]
df.reset_index().to_feather(f'all_clwc_{str(ds1.time.values)[:10]}.arrow')

In [ ]:
table = pa.Table.from_arrays([ ds1.latitude.values, ds1.longitude.values, out ], names=['lat','lon','clwp'])
pq.write_table(table, f'all_clwc.parquet')